<a href="https://colab.research.google.com/github/Musalinux/LinkedIn-DM-Automation/blob/main/LinkedIn_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🔹 Step 1: Upload LinkedIn connections exported data

In [ ]:
from google.colab import files
files.upload()

## 🔹 Step 2: Install Dependencies

In [ ]:
!pip install selenium beautifulsoup4 pandas requests openai playwright
!playwright install


## 🔹 Step 3: Import Libraries

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openai
import time

## 🔹 Step 4: Load LinkedIn Connections
Manually export your LinkedIn connections as a CSV and upload to Colab.
Then let's load this CSV

In [ ]:
from google.colab import files

uploaded = files.upload()  # Upload Connections.csv manually

df = pd.read_csv("Connections.csv")  # Load LinkedIn exported CSV

# Keep only necessary columns
df = df[['First Name', 'Last Name', 'Company', 'Profile URL']]

print(df.head())  # Display first 5 rows


## 🔹 Step 5: Scrape Company Career Pages
We check if the company has job openings automatically.



### A. Find the Career Page

In [ ]:
def check_company_jobs(company_name):
    search_url = f"https://www.google.com/search?q={company_name}+careers+jobs"

    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    for link in soup.find_all("a", href=True):
        if "careers" in link["href"] or "jobs" in link["href"]:
            return link["href"]
    return None


### B. Scrape Job Listings

In [ ]:
def scrape_job_listings(careers_page):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(careers_page, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    jobs = []
    for job in soup.find_all("a"):
        title = job.get_text()
        link = job["href"]
        if "job" in title.lower():
            jobs.append((title, link))

    return jobs

## 🔹 Step 6: Integrate ChatGPT/DeepSeek for Personalized Messages
We use OpenAI API (ChatGPT) or DeepSeek API to generate highly personalized messages.

### A. Setup API Key
Using OpenAI (ChatGPT)
1. Get your OpenAI API Key from platform.openai.com
2. Set it in Colab:

In [ ]:
openai.api_key = "YOUR_OPENAI_API_KEY"

### B. Generate Personalized Messages
Modify the function to use ChatGPT or DeepSeek for better message generation.

In [ ]:
def generate_message(person_name, company_name, job_available, profile_url):
    prompt = f"""
    You are an AI assistant helping job seekers. Write a highly professional, warm, and engaging LinkedIn outreach message.

    The person is {person_name}, working at {company_name}.
    - If a job opening exists, ask for a referral.
    - If no openings, introduce yourself and ask about hidden opportunities.

    Keep it **concise, polite, and natural**.

    Profile URL: {profile_url}
    """

    # Using OpenAI ChatGPT
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Use "gpt-3.5-turbo" if gpt-4 isn't available
        messages=[{"role": "system", "content": "You are a helpful job-seeking assistant."},
                  {"role": "user", "content": prompt}]
    )

    return response["choices"][0]["message"]["content"]

# Test Example
msg = generate_message("Alice", "Skyscanner", True, "https://linkedin.com/in/alice")
print(msg)


## 🔹 Step 7: Process All Connections
Now, we loop through each LinkedIn connection and generate messages.

In [ ]:
output_data = []

for index, row in df.iterrows():
    person_name = row['First Name']
    company_name = row['Company']
    profile_url = row['Profile URL']

    print(f"Checking jobs at {company_name}...")

    careers_page = check_company_jobs(company_name)

    if careers_page:
        jobs = scrape_job_listings(careers_page)
        job_available = len(jobs) > 0
    else:
        job_available = False

    message = generate_message(person_name, company_name, job_available, profile_url)
    output_data.append([company_name, person_name, message, "Yes" if job_available else "No"])

    time.sleep(2)  # Prevent rate limits

# Save results
output_df = pd.DataFrame(output_data, columns=["Company Name", "Person Name", "Message", "Vacancy Yes/No"])
output_df.to_csv("linkedin_outreach.csv", index=False)

print("Messages saved successfully!")


## 🔹 Step 8: Download Final Results
To download the final output CSV:

In [ ]:
from google.colab import files

files.download("linkedin_outreach.csv")

## ✅ Summary of Steps
### 1️⃣ Setup
1. Install selenium, beautifulsoup4, openai, pandas
2. Upload LinkedIn CSV to Colab

### 2️⃣ Extract LinkedIn Data
Load filtered contacts from CSV

### 3️⃣ Scrape Career Pages
1. Find company career page
2. Check if job openings exist

### 4️⃣ Generate Personalized Messages
Use ChatGPT/DeepSeek API to create high-quality outreach messages

### 5️⃣ Save & Download
1. Store all messages in CSV
2. Download the results